In [ ]:
# Import dependencies
import requests
import pandas as pd
import numpy as np

In [ ]:
# Base url for the api call
url = "https://clinicaltrials.gov/api/query/study_fields"

In [ ]:
# Headers for json call
headers = {'Content-Type': 'application/json', 
           'Accept': 'application/json'}

In [ ]:
# Extract data and create DataFrame
def create_df(x):

    # Create a list to hold the dictionaries from the json response
    sourced_data = []

    # Set a counter to increase the ranks with each iteration
    # Table search criteria limited to 'breast cancer' and from 01/01/2018
    counter = 0

    while True:
        # Define the parameters of the url
        params = {'expr': 'breast cancer AND AREA[StartDate]RANGE[01/01/2018,MAX]', 
                'fields': ','.join(x), 
                'min_rnk': 1 + 1000 * counter,  
                'max_rnk': 1000 + 1000 * counter, 
                'fmt': 'json'}

        # Make the call 
        response = requests.get(url, 
                                headers = headers, 
                                params = params)

        

        # Increment the counter
        counter += 1
        
        # Check to see if more data was returned for the current response
        if response.json()['StudyFieldsResponse']['NStudiesReturned'] == 0:
            # If there was no new data returned, break out of the while loop
            break

        # Add the data from the api call to the sourced_data list
        sourced_data += response.json()['StudyFieldsResponse']['StudyFields']
    return sourced_data

In [ ]:
# Define function to clean DataFrame removing unnecessary characters for analysis

def clean_data(df, fields):
    # Change datatype of the values, preparing for reg expression
    df[fields] = df[fields].astype(str)

    # Remove any unnecessary characters and turn blank values with NaN (null)
    for i in fields:
        df[i] = df[i].str.replace("^\[.|.\]$|'","")
        df[i] = df[i].replace('',np.nan)
        

In [ ]:
# Define function to clean DataFrame removing unnecessary characters for analysis

def clean_data_years(df, fields):
    # Change datatype of the values, preparing for reg expression
    df[fields] = df[fields].astype(str)

    # Remove any unnecessary characters and turn blank values with NaN (null)
    for i in fields:
        df[i] = df[i].str.replace('Years', '')
        df[i] = df[i].str.replace('Year', '')

In [ ]:
# List of fields for table A
table_a_cols = ['OrgStudyId',
               'BriefTitle',
               'StartDate',
               'CompletionDate',
               'OverallStatus',
               'StudyType']

In [ ]:
# Call the function the create DataFrame for table A
registration_df = pd.DataFrame(create_df(table_a_cols))
registration_df.set_index('Rank', inplace=True)

In [ ]:
# Clean DataFrame for table A
clean_data(registration_df, table_a_cols)

In [ ]:
# Renaming the columns for table A and convertion to datetime and removing duplicates
registration_df.columns = ['ID', 'Title', 'Start_Date', 'Completion_Date', 'Status', 'Study_Type']
registration_df.index.names = ['Index']
registration_df['Start_Date'] = pd.to_datetime(registration_df['Start_Date'])
registration_df['Completion_Date'] = pd.to_datetime(registration_df['Completion_Date'])
registration_df.drop_duplicates(subset=['ID'], keep='first')
registration_df

In [ ]:
# List of fields for table B
table_b_cols = ['OrgStudyId',
               'Gender',
               'MinimumAge',
               'HealthyVolunteers']

In [ ]:
# Call the function the create DataFrame for table B
participant_df = pd.DataFrame(create_df(table_b_cols))
participant_df.set_index('Rank', inplace=True)

In [ ]:
# Clean DataFrame for table B
clean_data(participant_df, table_b_cols)
clean_data_years(participant_df, table_b_cols)

In [ ]:
# Renaming the columns for table B and getting rid of rows
participant_df.columns = ['ID', 'Gender', 'Minimum_Age', 'Healthy_Volunteers']
participant_df.index.names = ['Index']
participant_df = participant_df[participant_df['Minimum_Age'].str.contains('Months') == False]
participant_df.drop_duplicates(subset=['ID'], keep='first')
participant_df

In [ ]:
# List of fields for table C
table_c_cols = ['OrgStudyId',
               'IsFDARegulatedDrug',
               'IsFDARegulatedDevice',
               'ResponsiblePartyType']

In [ ]:
# Call the function the create DataFrame for table C
study_details_df = pd.DataFrame(create_df(table_c_cols))
study_details_df.set_index('Rank', inplace=True)

In [ ]:
# Clean DataFrame for table C
clean_data(study_details_df, table_c_cols)

In [ ]:
# Renaming the columns for table C and removing duplicates
study_details_df.columns = ['ID', 'FDA_Regulated_Drug', 'FDA_Regulated_Device', 'Responsible_Party']
study_details_df.index.names = ['Index']
study_details_df.drop_duplicates(subset=['ID'], keep='first')
study_details_df

In [ ]:
# List of fields for table d
table_d_cols = ['OrgStudyId',
          'StudyType',
          'ArmGroupType',
          'InterventionType',
          'DesignInterventionModel',
          'DesignObservationalModel',
          'TargetDuration',
          'SamplingMethod',
          'Phase']

# Call function to create DataFrame for table D
study_method_df = pd.DataFrame(create_df(table_d_cols))
study_method_df.set_index('Rank',inplace=True)

In [ ]:
# Clean DataFrame for table c
clean_data(study_method_df, table_d_cols)

In [ ]:
# Renaming the columns for table D and removing duplicates
study_method_df.columns = ['ID', 'Study_Type', 'Arm_Group_Type', 'Intervention_Type', 'Interventional_Study_Model', 'Observational_Study_Model', 'Target_Duration', 'Sampling_Method', 'Phase']
study_method_df.index.names = ['Index']
study_method_df.drop_duplicates(subset=['ID'], keep='first')
study_method_df

In [ ]:
# List of fields for table e and export dataset into DataFrame
table_e_cols = ['OrgStudyId',
          'EnrollmentCount',
          'PrimaryOutcomeMeasure',]
free_text_df = pd.DataFrame(create_df(table_e_cols))
free_text_df.set_index('Rank', inplace=True)


In [ ]:
clean_data(free_text_df, table_e_cols)

In [ ]:
# Renaming the columns for table E and removing duplicates
free_text_df.columns = ['ID', 'Enrollment_Count', 'Primary_Outcome_Measure']
free_text_df.index.names = ['Index']
free_text_df.drop_duplicates(subset=['ID'], keep='first')
free_text_df

In [ ]:
# Export DataFrames as csv files 
study_method_df.to_csv('Tables/study_method_df.csv',index=False)
free_text_df.to_csv('Tables/free_text_df.csv', index=False)
registration_df.to_csv('Tables/registration_df.csv', index=False)
participant_df.to_csv('Tables/participant_df.csv', index=False)
study_details_df.to_csv('Tables/study_details_df.csv', index=False)